# Nmap 基礎語法指南

Nmap（Network Mapper）是一個強大的網路探索和安全審計工具。本指南將介紹 nmap 的基本語法和常用功能。

## 目錄
1. [基本語法](#基本語法)
2. [目標指定](#目標指定)
3. [主機發現](#主機發現)
4. [掃描類型](#掃描類型)
5. [連接埠指定](#連接埠指定)
6. [服務和版本偵測](#服務和版本偵測)
7. [輸出格式](#輸出格式)
8. [實用範例](#實用範例)

## 1. 基本語法

Nmap 的基本語法格式：
```
nmap [掃描類型] [選項] {目標規格}
```

### 最簡單的掃描

## 學習目標（這份講義你會學到）

- 了解 nmap 的基本用途與安全考量
- 能用最常見的 3 個步驟開始掃描：主機發現、埠掃描、版本偵測
- 實作並解讀簡單結果（範例可在 macOS / Linux 上執行）
- 知道如何把掃描輸出存成檔案以便後續分析

> 前置需求：具備基本的命令列操作經驗，並且只對授權的目標執行掃描。

In [ ]:
nmap 192.168.1.1          # 掃描單一 IP
nmap google.com           # 掃描域名

## 2. 目標指定方式

### 2.1 單一目標

In [ ]:
nmap 192.168.1.1          # IP 地址
nmap www.example.com      # 域名

### 2.2 多個目標

In [ ]:
nmap 192.168.1.1 192.168.1.5 192.168.1.10    # 多個 IP
nmap 192.168.1.1-10                           # IP 範圍
nmap 192.168.1.0/24                           # CIDR 表示法（整個子網）
nmap 192.168.1.*                              # 萬用字元

### 2.3 從檔案讀取目標

In [ ]:
nmap -iL targets.txt      # 從檔案讀取目標列表

### 2.4 排除目標

In [ ]:
nmap 192.168.1.0/24 --exclude 192.168.1.1     # 排除特定 IP
nmap 192.168.1.0/24 --excludefile exclude.txt # 從檔案排除

## 3. 主機發現

### 為什麼主機發現很重要？
在進行連接埠掃描之前，先找出網路中哪些主機是活躍的可以：
- 節省掃描時間
- 減少不必要的網路流量
- 避免掃描離線或不存在的主機

下面我們示範三個安全且常用的步驟，對初學者來說，先從主機發現開始：
1. 使用最簡單的主機發現（-sn）
2. 在防火牆情況下跳過 ping（-Pn）
3. 使用不同的探測類型（ICMP/TCP/UDP）來提高命中率


In [ ]:
# 1) 主機發現：快速掃描整個子網的活躍主機（只檢測是否存活，不掃描埠）
nmap -sn 192.168.1.0/24

In [ ]:
# 2) 在有可能被防火牆阻擋 ICMP 的情況下：跳過 ping，直接掃描埠（慎用，會增加流量）
nmap -Pn 192.168.1.1

In [ ]:
# 3) 自訂探測類型：多種探測提高發現率（ICMP/TCP/UDP 範例）
nmap -PE 192.168.1.1   # ICMP Echo
nmap -PS22,80,443 192.168.1.1   # TCP SYN 到常見埠
nmap -PU53 192.168.1.1   # UDP 到 DNS 埠（53）

## 4. 掃描類型
### 3.1 TCP 掃描

In [ ]:
# TCP SYN 掃描（預設，需要 root 權限）
nmap -sS 192.168.1.1      # 半開連接掃描，速度快且隱蔽

# TCP Connect 掃描
nmap -sT 192.168.1.1      # 完整 TCP 連接，無需 root 權限

# TCP ACK 掃描
nmap -sA 192.168.1.1      # 用於檢測防火牆規則

# TCP Window 掃描
nmap -sW 192.168.1.1      # 基於 TCP 視窗大小的掃描（古老方式）

### 3.2 UDP 掃描
nmap -sU 192.168.1.1      # UDP 掃描，速度較慢但重要

### 3.3 其他掃描類型
nmap -sN 192.168.1.1      # TCP Null 掃描
nmap -sF 192.168.1.1      # TCP FIN 掃描
nmap -sX 192.168.1.1      # TCP Xmas 掃描（FIN, PSH, URG 旗標）

## 4. 連接埠指定

### 4.1 指定連接埠範圍

In [ ]:
nmap -p 22 192.168.1.1           # 單一連接埠
nmap -p 22,80,443 192.168.1.1    # 多個連接埠
nmap -p 1-100 192.168.1.1        # 連接埠範圍
nmap -p- 192.168.1.1             # 所有連接埠 (1-65535)

### 4.2 常用連接埠選項
nmap -F 192.168.1.1              # 快速掃描（前 100 個常用連接埠）
nmap --top-ports 1000 192.168.1.1  # 掃描最常用的 1000 個連接埠

### 4.3 協議指定
nmap -p T:80,443 192.168.1.1     # TCP 連接埠
nmap -p U:53,161 192.168.1.1     # UDP 連接埠
nmap -pT:80,U:53 192.168.1.1     # 同時指定 TCP 和 UDP

## 5. 服務和版本偵測

### 6.1 服務偵測

In [ ]:
nmap -sV 192.168.1.1             # 偵測服務版本
nmap -sV --version-light 192.168.1.1  # 輕量級版本偵測
nmap -sV --version-all 192.168.1.1    # 嘗試所有探測

### 6.2 作業系統偵測
nmap -O 192.168.1.1              # 作業系統偵測
nmap -O --osscan-guess 192.168.1.1    # 積極猜測作業系統

### 6.3 腳本掃描
nmap -sC 192.168.1.1             # 使用預設腳本
nmap --script vuln 192.168.1.1   # 漏洞掃描腳本
nmap --script=ssl-enum-ciphers 192.168.1.1  # 特定腳本

## 7. 時間和效能調整

In [ ]:
### 7.1 掃描速度
nmap -T0 192.168.1.1             # 偏執模式（最慢）
nmap -T1 192.168.1.1             # 鬼祟模式
nmap -T2 192.168.1.1             # 禮貌模式
nmap -T3 192.168.1.1             # 正常模式（預設）
nmap -T4 192.168.1.1             # 積極模式
nmap -T5 192.168.1.1             # 瘋狂模式（最快）

### 7.2 自訂時間控制
nmap --min-rate 1000 192.168.1.1      # 最小封包傳送速率
nmap --max-rate 5000 192.168.1.1      # 最大封包傳送速率
nmap --scan-delay 1s 192.168.1.1      # 掃描延遲
nmap --max-retries 3 192.168.1.1      # 最大重試次數

## 8. 輸出格式

### 8.1 輸出到檔案
```bash
nmap -oN output.txt 192.168.1.1      # 正常輸出
nmap -oX output.xml 192.168.1.1      # XML 格式
nmap -oG output.gnmap 192.168.1.1    # Grepable 格式
nmap -oA basename 192.168.1.1        # 所有格式（.nmap, .xml, .gnmap）
```

### 8.2 詳細程度控制
```bash
nmap -v 192.168.1.1                  # 詳細模式
nmap -vv 192.168.1.1                 # 非常詳細模式
nmap -d 192.168.1.1                  # 除錯模式
nmap -dd 192.168.1.1                 # 更詳細的除錯
```

## 9. 進階與安全考量（初學者可先略過）

某些技巧屬於進階或具潛在攻擊性（例如分片、誘餌、來源埠偽裝、偽造 MAC 等）。
為了讓初學者專注於安全且合法的基本流程，這些內容已移到本文檔末尾的「附錄 - 進階指令」。
如果你已經熟悉基本掃描並且擁有明確授權，可以到附錄閱讀這些進階技術。

提示：初學者建議先練習主機發現、連接埠掃描與版本偵測三個步驟，之後再研究附錄的細節。

## 10. 常用組合指令（簡化）

這裡列出幾個對初學者最有用的安全組合指令：
- 只做主機發現並把結果存檔：`nmap -sn -oN hostlist.txt 192.168.1.0/24`
- 掃描常用埠並偵測版本：`nmap -sV --top-ports 1000 192.168.1.1`
- 全面快速檢查（只在授權情況下執行）：`nmap -A 192.168.1.1`

進階的隱蔽或繞過技巧（如分片/誘餌/偽造來源等）已移到附錄。

## 11. 實用腳本範例

### 11.1 常用 NSE 腳本類別
```bash
# 預設腳本（安全且實用）
nmap -sC 192.168.1.1

# 漏洞檢測腳本
nmap --script vuln 192.168.1.1

# 惡意軟體檢測
nmap --script malware 192.168.1.1

# 暴力破解腳本
nmap --script brute 192.168.1.1

# 服務發現腳本
nmap --script discovery 192.168.1.1
```

### 11.2 特定服務腳本
```bash
# HTTP 相關
nmap --script http-enum 192.168.1.1
nmap --script http-headers 192.168.1.1
nmap --script http-methods 192.168.1.1

# FTP 相關
nmap --script ftp-anon,ftp-bounce,ftp-proftpd-backdoor 192.168.1.1

# SSH 相關  
nmap --script ssh-hostkey,ssh-auth-methods 192.168.1.1

# DNS 相關
nmap --script dns-zone-transfer 192.168.1.1
```

## 12. 輸出解讀

### 12.1 連接埠狀態
- **open**: 連接埠開放且有服務在監聽
- **closed**: 連接埠關閉但可達
- **filtered**: 連接埠被防火牆過濾，無法確定狀態
- **unfiltered**: 連接埠可達但無法確定開放或關閉
- **open|filtered**: 無法確定是開放還是被過濾
- **closed|filtered**: 無法確定是關閉還是被過濾

### 12.2 服務偵測輸出範例
```
22/tcp   open  ssh     OpenSSH 7.4 (protocol 2.0)
80/tcp   open  http    Apache httpd 2.4.6
443/tcp  open  https   Apache httpd 2.4.6 ((CentOS) OpenSSL/1.0.2k-fips)
```

### 12.3 作業系統偵測範例
```
Running: Linux 3.X|4.X
OS CPE: cpe:/o:linux:linux_kernel:3 cpe:/o:linux:linux_kernel:4
OS details: Linux 3.2 - 4.9
```

## 13. 實際範例與練習

### 範例 1: 網路設備發現
目標：找出網路中的活躍設備

In [ ]:
# 範例 1: 探索本地網路中的活躍主機（可直接執行）
# 請將 192.168.1.0/24 替換為您的網段，例如 192.168.50.0/24

# 只做主機發現（不掃描埠）
nmap -sn 192.168.1.0/24

# 常用的快速埠掃描（只掃常見的 100 個埠）
nmap -F 192.168.1.1

# 若要把結果存成檔案：
nmap -oN hostlist.txt -sn 192.168.1.0/24

### 範例 2: 服務版本偵測

In [ ]:
# 範例 2: 詳細的服務版本偵測
echo "=== 服務版本偵測範例 ==="

# 掃描常用連接埠並偵測服務版本
# nmap -sV --version-intensity 5 -p 21,22,23,25,53,80,110,443,993,995 scanme.nmap.org

echo "這個指令會："
echo "1. 掃描指定的常用連接埠"
echo "2. 偵測服務版本（強度等級 5）"
echo "3. 嘗試識別服務的具體版本號"

echo ""
echo "=== 作業系統偵測 ==="
# nmap -O --osscan-guess scanme.nmap.org

echo "這會嘗試識別目標的作業系統"

### 範例 3: 腳本掃描

In [ ]:
# 範例 3: 使用 NSE 腳本進行深度掃描
echo "=== NSE 腳本掃描範例 ==="

# 使用預設腳本掃描
# nmap -sC scanme.nmap.org

echo "預設腳本包括："
echo "- 服務枚舉"
echo "- 安全檢查"
echo "- 版本偵測增強"

echo ""
echo "=== 漏洞掃描 ==="
# nmap --script vuln scanme.nmap.org

echo "漏洞掃描會檢查："
echo "- 已知的CVE漏洞"
echo "- 常見的錯誤配置"
echo "- 弱密碼或預設憑證"

echo ""
echo "=== HTTP 服務詳細掃描 ==="
# nmap --script http-enum,http-headers,http-methods,http-title -p 80,443 scanme.nmap.org

echo "HTTP 掃描包括："
echo "- 目錄和檔案枚舉"
echo "- HTTP 標頭分析"
echo "- 支援的 HTTP 方法"
echo "- 網頁標題獲取"

## 14. 安全注意事項

### 14.1 法律和道德考量
- ⚠️ **僅在您擁有或獲得明確授權的系統上使用 nmap**
- ⚠️ **未經授權的網路掃描可能違法**
- ⚠️ **某些掃描技術可能被視為攻擊行為**

### 14.2 最佳實踐
1. **獲得書面授權**：在掃描任何網路之前
2. **使用適當的時機**：避免在業務高峰時段掃描
3. **限制掃描範圍**：只掃描必要的目標和連接埠
4. **記錄所有活動**：保持掃描日誌用於審計

### 14.3 防止被發現
```bash
# 使用較慢的掃描速度
nmap -T1 target

# 使用隨機化選項
nmap --randomize-hosts --scan-delay 5s target

# 分散掃描時間
# 將大範圍掃描分成多個小批次
```

## 15. 故障排除

### 15.1 常見問題
```bash
# 權限不足
sudo nmap -sS target          # 需要 root 權限進行 SYN 掃描

# 防火牆阻擋
nmap -Pn target              # 跳過 ping，直接掃描連接埠

# 掃描速度太慢
nmap -T4 --min-rate 1000 target  # 提高掃描速度

# 連接逾時
nmap --host-timeout 5m target    # 增加主機逾時時間
```

### 15.2 除錯技巧
```bash
# 增加詳細輸出
nmap -v -d target            # 詳細模式 + 除錯模式

# 封包追蹤
nmap --packet-trace target   # 顯示發送和接收的封包

# 檢查路由
nmap --traceroute target     # 顯示到目標的路由路徑
```

## 16. 總結

### 16.1 重點回顧
- **基本語法**：`nmap [選項] {目標}`
- **主要掃描類型**：`-sS` (SYN), `-sT` (TCP), `-sU` (UDP)
- **重要選項**：`-sV` (版本), `-O` (作業系統), `-sC` (腳本)
- **輸出控制**：`-v` (詳細), `-oA` (所有格式)

### 16.2 學習建議
1. **從基本開始**：先掌握基本的掃描指令
2. **實際練習**：使用 `scanme.nmap.org` 進行練習
3. **閱讀文檔**：`man nmap` 和官方文檔
4. **探索腳本**：學習 NSE 腳本的使用

### 16.3 進階學習方向
- **自訂 NSE 腳本開發**
- **與其他工具整合** (如 Metasploit)
- **自動化掃描腳本撰寫**
- **大規模網路掃描策略**

---

**記住：nmap 是一個強大的工具，請負責任地使用！**

# 附錄 - 進階指令（僅供學術與授權測試）

> 重要聲明：以下內容屬於進階且可能被視為攻擊性的技術。請僅在你擁有明確書面授權的情況下使用。未經授權使用可能違法，並造成嚴重後果。

## 分片技術
```bash
nmap -f 192.168.1.1               # 使用小分片封包
nmap -ff 192.168.1.1              # 使用更小的分片
nmap --mtu 24 192.168.1.1         # 自訂 MTU 大小
```

## 誘餌掃描
```bash
nmap -D RND:10 192.168.1.1        # 使用 10 個隨機誘餌
nmap -D decoy1,decoy2,ME,decoy3 192.168.1.1  # 指定誘餌 IP
```

## 來源連接埠偽裝
```bash
nmap --source-port 53 192.168.1.1    # 使用 DNS 連接埠作為來源
nmap -g 53 192.168.1.1                # 同上（簡短格式）
```

## 其他繞過技術
```bash
nmap --data-length 25 192.168.1.1    # 添加隨機資料到封包
nmap --randomize-hosts 192.168.1.0/24  # 隨機化目標順序
nmap --spoof-mac 0 192.168.1.1        # 偽造 MAC 地址
```

## 隱蔽掃描與組合範例
```bash
# 慢速鬼祟掃描
nmap -sS -T1 -f -D RND:10 192.168.1.1

# SYN 掃描 + 版本偵測 + 誘餌
nmap -sS -sV -D RND:5 -f --randomize-hosts 192.168.1.0/24
```

---

再次強調：僅在授權情況下閱讀與實驗，並保存所有測試紀錄以備查驗。